In [ ]:
from ml.generator.lr12 import V1, V1Setting

v1 = V1(V1Setting())

In [ ]:
from ml.feature_engineering.factory import MLXFeaturesFactory
from ml.feature_engineering.derivative_features_extractor import Extractor, ExtractorSetting
from ml.feature_engineering.factors.f_mlp import MAD, MADSetting, BLG, BLGSetting, UUP, UUPSetting
from ml.feature_engineering.factors.f_ind import RSI, RSISetting, OBVMAD, OBVMADSetting

mad = MAD(MADSetting(fast=5, slow=21))
mad1 = MAD(MADSetting(fast=21, slow=89))
blg = BLG(BLGSetting(period=5))
blg1 = BLG(BLGSetting(period=13))
blg2 = BLG(BLGSetting(period=21))
blg3 = BLG(BLGSetting(period=89))
uup = UUP(UUPSetting(period=5))
uup1 = UUP(UUPSetting(period=13))
uup2 = UUP(UUPSetting(period=21))
uup3 = UUP(UUPSetting(period=89))

rsi = RSI(RSISetting(period=21))
rsi1 = RSI(RSISetting(period=55))

obv = OBVMAD(OBVMADSetting(fast=5, slow=21))
obv1 = OBVMAD(OBVMADSetting(fast=21, slow=89))

factors = [mad, blg, uup, mad1, blg1, uup1, blg2, uup2, blg3, uup3, rsi, rsi1, obv, obv1]

extractor=Extractor(ExtractorSetting(id="v1", roll_max_timeshift=21, roll_min_timeshift=5, n_jobs=95))
features_factory = MLXFeaturesFactory()

features_factory.set_extractor(extractor)

features_factory.add_factor(factors=factors)


df = features_factory.factor_extract(v1.df)

df.dropna(inplace=True)

df

In [ ]:
dfe_data = {
    mad.id(): df[mad.id()], blg.id(): df[blg.id()], uup.id(): df[uup.id()],
    mad1.id(): df[mad1.id()], blg1.id(): df[blg1.id()], uup1.id(): df[uup1.id()],
    blg2.id(): df[blg2.id()], uup2.id(): df[uup2.id()], blg3.id(): df[blg3.id()], uup3.id(): df[uup3.id()],
    rsi.id(): df[rsi.id()], rsi1.id(): df[rsi1.id()], obv.id(): df[obv.id()], obv1.id(): df[obv1.id()]
}

X = features_factory.derivative_features(dfe_data, df["n_log_return"])

X

In [ ]:

uuid = "a7856b39-2e54-4282-aead-96d738cbd0d4"


feature = X.join(df["y"])

uid = features_factory.save_derivative_features(feature)

print(uid)

feature

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

num_train = round(len(feature)*0.7)

data_train = feature.iloc[:num_train, :]
data_test = feature.iloc[num_train:, :]


x_train = data_train.drop("y", axis=1).values
y_train = data_train['y']

x_test = data_test.drop("y", axis=1).values
y_test = data_test['y']

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


model = SVC(C=1, kernel='rbf')
model.fit(x_train, y_train)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

train_score = model.score(x_train,y_train)
print("train score",train_score)

test_score = model.score(x_test,y_test)
print("test score",test_score)

# report
report = classification_report(y_test, y_test_pred, labels=[-1, 0, 1], target_names=['short', 'no', 'long'])
print("report", report)
